## Testing Adjoint Code Generation

In [ ]:
import ast
import inspect
import ad_transformer
import expr_transformer
from draw_ast import *
import simple_expressions
import importlib

importlib.reload(ad_transformer)
importlib.reload(expr_transformer)
importlib.reload(simple_expressions)
from ad_transformer import *
from expr_transformer import *
from simple_expressions import *

In [ ]:
# 1) Matrix-matrix fma
expr = "Y = A @ X + C"
ad_expr = transform_expr(expr)
print(ad_expr)

In [ ]:
# 2) Sum of Squared Residuals
expr2 = "R = (y - X @ b).T @ (y - X @ b)"
print(transform_expr(expr2))

In [ ]:
# 3) Generalized Least Squares
expr3 = "b = numpy.linalg.inv(X.T @ numpy.linalg.inv(M) @ X) @ X.T @ numpy.linalg.inv(M) @ y"
print(transform_expr(expr3))

In [ ]:
import numpy
X = numpy.eye(3)
X_a = numpy.zeros((3,3))
y = np.ones((3,)) # column vec
y_a = np.zeros((3,))
b_a = numpy.array([0, 1, 0])

v0 = X.T
v1 = v0 @ X
v2 = numpy.linalg.inv(v1)
v3 = X.T
v4 = v2 @ v3
b = v4 @ y
v0_a = X_a.T
v1_a = numpy.zeros(v1.shape)
v2_a = numpy.zeros(v2.shape)
v3_a = X_a.T
v4_a = numpy.zeros(v4.shape)
y_a += v4.T @ b_a
v4_a += b_a @ y.T
v3_a += v2.T @ v4_a
v2_a += v4_a @ v3.T
v1_a -= numpy.linalg.inv(v1).T @ (v2_a @ numpy.linalg.inv(v1).T)
X_a += v0.T @ v1_a
v0_a += v1_a @ X.T

X_a

In [ ]:
# 2nd example: matrix form of quadratic
expr3 = "y = x.T @ A @ x + b.T @ x - c"
print(transform_expr(expr3))

In [ ]:
transformer = AdjointNodeTransformer()
tree = ast.parse(inspect.getsource(simple_exp))
newAST = transformer.visit(tree) 

In [ ]:
print(ast.dump(newAST, indent=4))

In [ ]:
newAST = ast.fix_missing_locations(newAST)
print(ast.unparse(newAST))

In [ ]:
import numpy as np
A = 1.5 * np.ones((2,2))
B = 2.5 * np.ones((2,2))
C = 3.5 * np.ones((2,2))
A_a = np.zeros((2,2))
B_a = np.zeros((2,2))
C_a = np.zeros((2,2))

exec(compile(newAST, filename="<ast>", mode="exec", optimize=1)) # compile and execute to make it accessible

result, dfdA, dfdB, dfdC = simple_exp_ad(A, B, C, A_a, B_a, C_a)
print("primal result:\n {r}\n df/dA:\n {a}\n df/dB:\n {b}\n df/dC:\n {c}".format(r=result, a=dfdA, b=dfdB, c=dfdC))

In [ ]:
# transformer2 = AdjointNodeTransformer()
# tree2 = ast.parse(inspect.getsource(multiline_BinOps))
# newAST2 = transformer2.visit(tree2)
# newAST2 = ast.fix_missing_locations(newAST2)
# print(ast.unparse(newAST2))

In [ ]:
# A = np.ones((2,2))
# B = np.ones((2,2))
# C = -1.0 * np.ones((2,2))
# A_a = np.zeros((2,2))
# B_a = np.zeros((2,2))
# C_a = np.zeros((2,2))
# exec(compile(newAST2, filename="<ast>", mode="exec"))
# multiline_BinOps_ad(A, B, C, A_a, B_a, C_a)

In [ ]:
new_test = transform(quadratic)
# transformer3 = AdjointNodeTransformer()
# test_src = ast.parse(getsource(test))
# new_test = transformer3.visit(test_src)
# print(ast.dump(new_test, indent=3))
print(new_test)

In [ ]:
# print(ast.dump(ast.parse(transform(simple_exp)), indent=4))

In [ ]:
def quadratic_ad(A, x, b, A_a, x_a, b_a):
    v0 = A
    v1 = x
    v2 = b
    v3 = numpy.transpose(v1)
    v4 = v3 @ v0
    v5 = v4 @ v1
    v6 = numpy.transpose(v1)
    v7 = v6 @ v2
    v8 = v5 - v7
    v0_a = A_a
    v1_a = x_a
    v2_a = b_a
    v3_a = numpy.zeros(v3.shape)
    v4_a = numpy.zeros(v4.shape)
    v5_a = numpy.zeros(v5.shape)
    v6_a = numpy.zeros(v6.shape)
    v7_a = numpy.zeros(v7.shape)
    v8_a = numpy.ones(v8.shape)
    v7_a += -1.0 * v8_a
    v5_a += 1.0 * v8_a
    v2_a += v6 * v7_a
    v6_a += v2 * v7_a
    v1_a += v4 * v5_a
    v4_a += v1 * v5_a
    v0_a += v3 * v4_a
    v3_a += v0 * v4_a
    return (v8, v0_a, v1_a, v2_a)

In [ ]:
# exec(compile(new_test, filename="<ast>", mode="exec"))
shape_A = (1,1)
shape_x = (1,1)
A =  numpy.ones(shape_A)
A_a = numpy.zeros(shape_A)
x = 2 * numpy.ones(shape_x)
x_a = numpy.zeros(shape_x)
b = numpy.zeros(shape_x)
b_a = numpy.zeros(shape_x)
quadratic_ad(A, x, b, A_a, x_a, b_a)